In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from river import drift
from itertools import product
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore')

In [43]:
''' Leitura do arquivo com as funções necessárias pro pré-processamento e processamento do dados '''
# json_processed_data(folder_path), gpx_processed_data(data_folder_path) e fit_processed_data(data_folder_path)
%run C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts_prediction/feature_engineering_act_functions.ipynb

In [44]:
''' Contador de arquivos dentro da pasta '''

# Caminho da pasta com as pastas contendo as atividades de cada atleta
data_folders_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities'

folders_count = 0

for file in os.listdir(data_folders_path):
    folders_count += 1 

print(folders_count)

7


In [45]:
# Criação do Dataframe que terá os valores de cada atleta por id
df_athletes_activities = pd.DataFrame()

# Número do primeiro id (Se começar com 0, mudar para n = 0)
n = 1

# Iteração para criar o Dataframe com os valores de cada atleta por id
for athlete_id in range(n, folders_count + n):

    # Nome do arquivo com os dados do atleta
    athlete_foldername = f'athlete{athlete_id}'

    # Junção da pasta com as pastas contendo as atividades de cada atleta e o nome da pasta com os dados do atleta por id
    data_folder_path = os.path.join(data_folders_path, athlete_foldername)

    # Pegando a primeira atividade da pasta
    first_file = os.listdir(data_folder_path)[0]
    
    if first_file.endswith('.gpx'):
        # Criação do Dataframe a partir da função de processamento de dados gpx
        df = gpx_processed_data(data_folder_path)

    elif first_file.endswith('.json'):
        # Criação do Dataframe a partir da função de processamento de dados gpx
        df = json_processed_data(data_folder_path)

    elif first_file.endswith('.fit'):
        # Criação do Dataframe a partir da função de processamento de dados gpx
        df = fit_processed_data(data_folder_path)
    
    df_athlete_activity = activities_time_frequency(df)

    # Adição de uma coluna athlete_id para identificar os dados de cada atleta
    df_athlete_activity.insert(0, 'athlete_id', athlete_id)

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities = pd.concat([df_athletes_activities, df_athlete_activity])

df_athletes_activities.reset_index(drop=True, inplace=True)

In [ ]:
# Criação do Dataframe que terá os valores de cada atleta por id
df_athletes_activities = pd.DataFrame()

# Número do primeiro id (Se começar com 0, mudar para n = 0)
n = 1

# Iteração para criar o Dataframe com os valores de cada atleta por id
for athlete_id in range(n, folders_count + n):

    # Nome do arquivo com os dados do atleta
    athlete_foldername = f'athlete{athlete_id}'

    # Junção da pasta com as pastas contendo as atividades de cada atleta e o nome da pasta com os dados do atleta por id
    data_folder_path = os.path.join(data_folders_path, athlete_foldername)

    for filename in os.listdir(data_folder_path):
        # Junta o nome do arquivo com o caminho da pasta
        full_path = os.path.join(data_folder_path, filename)
    
        if first_file.endswith('.gpx'):
            # Criação do Dataframe a partir da função de processamento de dados gpx
            df = gpx_processed_data(data_folder_path)

        elif first_file.endswith('.json'):
            # Criação do Dataframe a partir da função de processamento de dados gpx
            df = json_processed_data(data_folder_path)

        elif first_file.endswith('.fit'):
            # Criação do Dataframe a partir da função de processamento de dados gpx
            df = fit_processed_data(data_folder_path)
        
        df_athlete_activity = activities_time_frequency(df)

    # Adição de uma coluna athlete_id para identificar os dados de cada atleta
    df_athlete_activity.insert(0, 'athlete_id', athlete_id)

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities = pd.concat([df_athletes_activities, df_athlete_activity])

df_athletes_activities.reset_index(drop=True, inplace=True)

In [46]:
df_athletes_activities_new = df_athletes_activities.copy() 
df_athletes_activities_new[df_athletes_activities_new['athlete_id'] == 6]

,athlete_id,activity_date,activity_type,total_distance (km),total_time (min),pace (min/km),velocity (km/h),days_between_activities
4314,6,2023-07-22,Activity,5.62,36.0,6.40,9.37,0
4315,6,2023-07-23,Activity,5.87,31.0,5.28,11.36,0
4316,6,2023-07-24,Activity,0.00,62.0,inf,0.00,0
4317,6,2023-07-25,Activity,0.00,56.0,inf,0.00,0
4318,6,2023-07-26,Activity,0.00,59.0,inf,0.00,0
...,...,...,...,...,...,...,...,...
4522,6,2024-02-15,No Activity,NaN,NaN,NaN,NaN,7
4523,6,2024-02-16,Activity,0.00,53.0,inf,0.00,0
4524,6,2024-02-17,Activity,0.00,77.0,inf,0.00,0
4525,6,2024-02-18,No Activity,NaN,NaN,NaN,NaN,1


In [47]:
# Transformando o tipo da atividade para binário
df_athletes_activities_new['activity_type'] = (df_athletes_activities_new['activity_type'] == 'Activity') * 1

In [48]:
# Número de atividades dentro desse período de dias em uma janela deslizante, por atleta
# Quantidade de atividades num período específico de dias
activity_days = 7
df_athletes_activities_new['activities_weekly_frequency'] = [values for values in df_athletes_activities_new.groupby('athlete_id')['activity_type'].rolling(activity_days).sum()]
df_athletes_activities_new.dropna(subset= 'activities_weekly_frequency', inplace= True)

In [49]:
no_activity = df_athletes_activities_new[df_athletes_activities_new['activity_type'] == 0].index
df_athletes_activities_new.drop(no_activity, inplace=True)
df_athletes_activities_new.reset_index(drop=True, inplace=True)

df_athletes_activities_new.drop(['activity_type', 'days_between_activities'], axis=1, inplace= True)

In [50]:
# Fazendo a diferença entre a atividade seguinte e a anterior e preenchendo a primeira linha com 0
df_athletes_activities_new['days_since_last_activity'] = df_athletes_activities_new.groupby('athlete_id')['activity_date'].diff().dt.days.fillna(0)

In [51]:
df_athletes_activities_new

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),velocity (km/h),activities_weekly_frequency,days_since_last_activity
0,1,2020-07-16,3.61,23.0,6.38,9.40,3.0,0.0
1,1,2020-07-20,25.66,127.0,4.95,12.12,2.0,4.0
2,1,2020-07-22,25.54,149.0,5.83,10.28,3.0,2.0
3,1,2020-07-24,5.84,41.0,7.02,8.55,3.0,2.0
4,1,2020-07-25,4.28,32.0,7.47,8.03,4.0,1.0
...,...,...,...,...,...,...,...,...
886,7,2024-06-16,9.99,45.0,4.51,13.31,3.0,3.0
887,7,2024-06-18,5.30,33.0,6.22,9.64,3.0,2.0
888,7,2024-06-21,12.10,62.0,5.12,11.71,3.0,3.0
889,7,2024-06-26,4.08,22.0,5.40,11.12,2.0,5.0


In [52]:
df_athletes_activities_new['physical_activity_churn'] = (df_athletes_activities_new['days_since_last_activity'] > 7) * 1
df_athletes_activities_new

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),velocity (km/h),activities_weekly_frequency,days_since_last_activity,physical_activity_churn
0,1,2020-07-16,3.61,23.0,6.38,9.40,3.0,0.0,0
1,1,2020-07-20,25.66,127.0,4.95,12.12,2.0,4.0,0
2,1,2020-07-22,25.54,149.0,5.83,10.28,3.0,2.0,0
3,1,2020-07-24,5.84,41.0,7.02,8.55,3.0,2.0,0
4,1,2020-07-25,4.28,32.0,7.47,8.03,4.0,1.0,0
...,...,...,...,...,...,...,...,...,...
886,7,2024-06-16,9.99,45.0,4.51,13.31,3.0,3.0,0
887,7,2024-06-18,5.30,33.0,6.22,9.64,3.0,2.0,0
888,7,2024-06-21,12.10,62.0,5.12,11.71,3.0,3.0,0
889,7,2024-06-26,4.08,22.0,5.40,11.12,2.0,5.0,0


In [53]:
df_athletes_activities_new.groupby('athlete_id')[['total_distance (km)', 'total_time (min)', 'pace (min/km)', 'velocity (km/h)']].describe()

total_distance (km)                                           \
                         count         mean           std   min     25%   
athlete_id                                                                
1                        111.0    23.890541     18.372118  2.02  5.7500   
2                         61.0     7.713770      4.649057  0.82  5.0300   
3                         94.0     6.902021      2.593302  0.34  5.2725   
4                        148.0     5.616081      2.152384  3.20  5.0100   
5                        352.0     9.237727      6.114741  0.00  5.0000   
6                         81.0  1456.228642   2400.153498  0.00  0.0000   
7                         44.0  6645.820455  15078.104301  0.00  4.3800   

                                       total_time (min)              ...  \
              50%        75%       max            count        mean  ...   
athlete_id                                                           ...   
1           23.46    35.1450     85.73            111.0  103.252252  ...   
2            5.20    10.1200     18.84             61.0   43.672131  ...   
3            6.60     8.6800     14.60             94.0   36.340426  ...   
4            5.01     5.0300     21.24            148.0   35.763514  ...   
5            8.04    13.0125     42.46            352.0   51.664773  ...   
6            0.00  5320.0500   5826.35             81.0   60.987654  ...   
7            7.58  1338.1600  79726.55             44.0   36.227273  ...   

           pace (min/km)         velocity (km/h)                              \
                     75%     max           count          mean           std   
athlete_id                                                                     
1                 6.5300    9.76           111.0     12.506667      3.923256   
2                 6.9200   41.68            61.0     10.131639      3.161112   
3                 5.5000    6.88            94.0     11.482660      0.852493   
4                 6.3825    7.14           148.0      9.610405      0.403614   
5                 7.0800  155.00           352.0     10.268949      3.738358   
6                    NaN     inf            81.0   2290.463704   3988.958065   
7                 5.4525     inf            44.0  10376.674545  23348.018833   

                                                        
             min      25%     50%        75%       max  
athlete_id                                              
1           6.15   9.1900  12.040    14.6050     22.13  
2           1.44   8.6700   9.180     9.9500     17.11  
3           8.72  10.9025  11.495    12.0000     13.87  
4           8.40   9.3975   9.595     9.8425     11.02  
5           0.00   7.9475  11.620    13.4300     15.57  
6           0.00   0.0000   0.000  5075.4500  14526.94  
7           0.00  11.0050  11.935  1864.8525  99658.18  

[7 rows x 32 columns]

In [55]:
df_athletes_activities_new[df_athletes_activities_new['pace (min/km)'] == 155]

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),velocity (km/h),activities_weekly_frequency,days_since_last_activity,physical_activity_churn
416,5,2021-11-18,0.2,31.0,155.0,0.39,3.0,1.0,0


In [54]:
df_atleta_6 = df_athletes_activities_new[df_athletes_activities_new['athlete_id'] == 6]

df_atleta_6

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),velocity (km/h),activities_weekly_frequency,days_since_last_activity,physical_activity_churn
766,6,2023-07-29,5324.09,36.0,0.01,8873.48,6.0,0.0,0
767,6,2023-08-03,0.00,62.0,inf,0.00,2.0,5.0,0
768,6,2023-08-04,0.00,78.0,inf,0.00,3.0,1.0,0
769,6,2023-08-05,5324.09,35.0,0.01,9127.01,3.0,1.0,0
770,6,2023-08-06,5329.22,63.0,0.01,5075.45,4.0,1.0,0
...,...,...,...,...,...,...,...,...,...
842,6,2024-02-07,0.00,66.0,inf,0.00,6.0,1.0,0
843,6,2024-02-08,0.00,124.0,inf,0.00,6.0,1.0,0
844,6,2024-02-16,0.00,53.0,inf,0.00,1.0,8.0,1
845,6,2024-02-17,0.00,77.0,inf,0.00,2.0,1.0,0
